In [1]:
from load_preprocess.functions import Data
import pandas as pd
import warnings
warnings.filterwarnings("ignore")


In [42]:
data = Data(path = 'data/')
df = data.create_df()

100%|██████████| 8/8 [00:23<00:00,  2.94s/it]


In [44]:
df.NAME_1.value_counts()

Île-de-France                 1
Provence-Alpes-Côte d'Azur    1
Aquitaine                     1
Limousin                      1
Auvergne                      1
Bourgogne                     1
Centre                        1
Franche-Comté                 1
Languedoc-Roussillon          1
Poitou-Charentes              1
Champagne-Ardenne             1
Nord-Pas-de-Calais            1
Pays de la Loire              1
Basse-Normandie               1
Bretagne                      1
Lorraine                      1
Rhône-Alpes                   1
Picardie                      1
Alsace                        1
Haute-Normandie               1
Midi-Pyrénées                 1
Name: NAME_1, dtype: int64

In [90]:
df_on = pd.read_excel(r'data/road_go_na_ru3g_spreadsheet.xlsx', sheet_name='Sheet 1', skiprows=8)
df_off = pd.read_excel(r'data/road_go_na_rl3g_spreadsheet.xlsx', sheet_name='Sheet 1', skiprows=8)

def clean_freight_df(df, on_load=True):
    
    df = df[['TIME', 'TIME.1', '2021']]
    
    if on_load:
        freight_type = "number_onload"
    else:
        freight_type = "number_offload"
        
    df.rename(columns={"TIME": "geo_code", "TIME.1": "geo_labels", "2021": freight_type}, inplace=True)
    df = df.iloc[1:]
    df_clean = df[df.geo_code.str.startswith("FR").fillna(False)]
    df_clean.reset_index(inplace=True, drop=True)
    
    df_clean[freight_type] = df_clean[freight_type].astype('str')
    df_clean[freight_type] = df_clean[freight_type].str.replace('\.0*$', '', regex=True)
    df_clean.loc[~(df_clean[freight_type].str.isdigit()), freight_type] = '0'
    df_clean[freight_type] = df_clean[freight_type].astype('int')
    
    return df_clean

def department_region_map(path, df_fr):
    df_dpts_region = pd.read_csv(path)
    df_dpts_region.rename(columns={"dep_name":"geo_labels", "region_name":"new_region_name", "old_region_name": "region"}, inplace=True)
    df_final = pd.merge(df_fr, df_dpts_region[["geo_labels", "region"]], how='left', on=['geo_labels'])
    df_final = df_final[~(df_final.geo_labels.str.endswith(" "))]
    return df_final

def merge_freight(path, df_on, df_off):
    
    df_onload_fr = clean_freight_df(df_on, on_load=True)
    df_offload_fr = clean_freight_df(df_off, on_load=False)

    df_fr = pd.merge(df_onload_fr, df_offload_fr, how='inner', on=['geo_code', 'geo_labels'])
    df_fr['total_load'] = df_fr.number_offload + df_fr.number_onload 
    
    df_fr['geo_labels'] = [c[0] for c in df_fr['geo_labels'].str.split("(")]
    df_fr = department_region_map(path, df_fr)

    return df_fr

path_region_dpt_map = 'data/departements-region.csv'
df_fr = merge_freight(path_region_dpt_map, df_on, df_off)

df_fr

,geo_code,geo_labels,number_onload,number_offload,total_load,region
0,FR101,Paris,5643,3505,9148,Île-de-France
1,FR102,Seine-et-Marne,47461,46147,93608,Île-de-France
2,FR103,Yvelines,15700,12510,28210,Île-de-France
3,FR104,Essonne,29109,30292,59401,Île-de-France
4,FR105,Hauts-de-Seine,11931,12161,24092,Île-de-France
...,...,...,...,...,...,...
91,FRL04,Bouches-du-Rhône,42245,45987,88232,Provence-Alpes-Côte d'Azur
92,FRL05,Var,19918,17422,37340,Provence-Alpes-Côte d'Azur
93,FRL06,Vaucluse,17470,17230,34700,Provence-Alpes-Côte d'Azur
94,FRM01,Corse-du-Sud,1131,925,2056,Corse
